In [1]:
import sys, os
sys.path.append('/Users/michaelgentnermac/Documents/ADL4CV/video_interpolation/')
import torch
import torch.nn as nn
from backprojection.bfgs import BFGSProjector
from interpolate_latent.functions import *
from models.wgan import Generator, Discriminator
import torchvision.transforms as transforms
netG = torch.load('/Users/michaelgentnermac/Documents/ADL4CV/final_models/wgan/g.pt',map_location=torch.device('cpu'))
netG = netG.double().cpu()
netD = torch.load('/Users/michaelgentnermac/Documents/ADL4CV/final_models/wgan/d.pt', map_location=torch.device('cpu'))
netD = netD.double().cpu()
netP = torch.load('/Users/michaelgentnermac/Documents/ADL4CV/final_models/backprojection/pnet_64.pt', map_location=torch.device('cpu'))
netP = netP.double().cpu()

netG = netG.eval()
netD = netD.eval()
netP = netP.eval()

In [2]:
def create_train_valid_test_split(data):
    train_data_len = int(data.shape[1]*0.8)
    valid_data_len = int(data.shape[1]*0.1)
    test_data_len = int(data.shape[1]*0.1)
    train_data = data[:, 0:train_data_len, :, :]
    valid_data = data[:, train_data_len:(valid_data_len+train_data_len), :, :]
    test_data = data[:, (train_data_len+valid_data_len):(train_data_len+valid_data_len+test_data_len), :, :]
    return train_data, valid_data, test_data

In [3]:
def create_folder(path):
    if not os.path.exists(path):
        os.makedirs(path)

In [4]:
data = np.load('/Users/michaelgentnermac/Documents/ADL4CV/mnist_test_seq_28.npy')
train_data, valid_data, test_data = create_train_valid_test_split(data)
toPil = transforms.ToPILImage()

### Sample images

In [14]:
num_images = 1

seq_length = test_data.shape[0]
num_seq = test_data.shape[1]

rand_imgs = np.random.randint(0, seq_length-1, num_images)
rand_seq = np.random.randint(0, num_seq-1, num_images)
rand_seq = [479]
rand_imgs = [6]

In [15]:
folder_name = '479'
projector = BFGSProjector(netG, 16, device='cpu', method = 'L-BFGS-B', loss = nn.BCELoss(), tol = None)

In [16]:
base_path = '/Users/michaelgentnermac/Documents/ADL4CV/results/Backprojection/'
create_folder(os.path.join(base_path, folder_name))
for i in range(num_images):
    
    img = torch.tensor(test_data[rand_imgs[i], rand_seq[i], :, :]).double().unsqueeze(0)
    #save original image
    path = os.path.join(base_path, folder_name, 'original')
    create_folder(path)
    toPil(img.squeeze().float()).save(os.path.join(path, str(i)+'.png'), 'PNG')
    
    #save bfgs image
    print("BFGS: Projecting image {}".format(i))
    path = os.path.join(base_path, folder_name, 'bfgs')
    create_folder(path)
    z = torch.tensor(projector.project(img.squeeze()).x)
    proj_img = netG(torch.tensor(z))
    toPil(proj_img.detach().squeeze().float()).save(os.path.join(path, str(i)+'.png'), 'PNG')
    
    #save pnet image
    print("PNET: Projecting image {}".format(i))
    path = os.path.join(base_path, folder_name, 'pnet')
    create_folder(path)
    pnet_img = netG(netP(img))
    toPil(pnet_img.detach().squeeze().float()).save(os.path.join(path, str(i)+'.png'), 'PNG')
    
    #save pnet+bfgs image
    print("PNET&BFGS: Projecting image {}".format(i))
    path = os.path.join(base_path, folder_name, 'pnet_bfgs')
    create_folder(path)
    z0 = netP(img).detach()
    z = torch.tensor(projector.project(img.squeeze(), x0=z0).x).double()
    pbfgs_img = netG(z)
    toPil(pbfgs_img.detach().squeeze().float()).save(os.path.join(path, str(i)+'.png'), 'PNG')
    

BFGS: Projecting image 0


/usr/local/Cellar/ipython/7.5.0/libexec/vendor/lib/python3.7/site-packages/ipykernel_launcher.py:16: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  app.launch_new_instance()


PNET: Projecting image 0
PNET&BFGS: Projecting image 0
